In [ ]:
import argparse
import json
import pickle
import pandas as pd
from nilearn.datasets import fetch_neurovault
import json
import os

In [3]:
#adapted from https://github.com/rmenuet/fmri_decoding/blob/master/Experiment/a1_collect.py

# ================================
# === FETCHING FROM NEUROVAULT ===
# ================================
def fetch_nv(repo, nv_file,
             download=False,
             max_images=None):
    """
    Loads neurovault into memory, either downloading it from the web-API or
    loading it from the disk.
    :param repo: str
        Path where the data is downloaded.
    :param nv_file: str
        Pickle file where the full data is saved
        (for faster loading than the fetch_neurovault).
    :param download: bool, default=False
        If True: the data is downloaded from the web-API.
    :param max_images: int, default=None
        Number of images to load from neurovalt.
    :return: Bunch
        A dict-like object containing the data from fMRIs fetched from
        Neurovault.
    """

    # Download and save to disk or load from disk

    if download:

        print("...Download from Neurovault API...")
        os.makedirs(repo, exist_ok=True)
        os.makedirs(nv_filepath, exist_ok=True)
        nv_file=os.path.join(nv_filepath, "nv_meta.p")
        
        neurovault = fetch_neurovault(max_images=max_images,
                                      collection_terms={},
                                      image_terms={},
                                      data_dir=repo,
                                      mode="download_new",
                                      verbose=2,
                                      collection_id= 4337)
        
        with open(nv_file, 'wb') as f:
            pickle.dump(neurovault, f)
    else:
       
        print("...Load pre-fetched data from Neurovault...")
        with open(nv_file, 'rb') as f:
            neurovault = pickle.load(f)

    n_fmri_dl = len(neurovault.images)

    print("  > Number of (down)loaded fmri =", n_fmri_dl)

    return neurovault

In [4]:
nv_path= "../../Data/neurovault/"
nv_filepath = "../../cache/"
download=True
max_images=10
neurovault = fetch_nv(nv_path, nv_filepath , download, max_images= max_images)


...Download from Neurovault API...

Dataset created in ../../Data/neurovault/neurovault

No image found on local disk.
Already fetched 1 image
Already fetched 2 images
Already fetched 3 images
Already fetched 4 images
Already fetched 5 images
Already fetched 6 images
Already fetched 7 images
Already fetched 8 images
Already fetched 9 images
Already fetched 10 images
  > Number of (down)loaded fmri = 10


In [5]:
neurovault

{'images': ['../../Data/neurovault/neurovault/collection_4337/image_69118.nii.gz',
  '../../Data/neurovault/neurovault/collection_4337/image_69119.nii.gz',
  '../../Data/neurovault/neurovault/collection_4337/image_69120.nii.gz',
  '../../Data/neurovault/neurovault/collection_4337/image_69121.nii.gz',
  '../../Data/neurovault/neurovault/collection_4337/image_69122.nii.gz',
  '../../Data/neurovault/neurovault/collection_4337/image_69123.nii.gz',
  '../../Data/neurovault/neurovault/collection_4337/image_69124.nii.gz',
  '../../Data/neurovault/neurovault/collection_4337/image_69125.nii.gz',
  '../../Data/neurovault/neurovault/collection_4337/image_69126.nii.gz',
  '../../Data/neurovault/neurovault/collection_4337/image_69127.nii.gz'],
 'images_meta': [{'url': 'https://neurovault.org/images/69118/',
   'id': 69118,
   'file': 'https://neurovault.org/media/images/4337/z_FACES.nii.gz',
   'collection': 'https://neurovault.org/collections/4337/',
   'collection_id': 4337,
   'file_size': 18541